In [1]:
import numpy as np
import tensorflow
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint

In [2]:
train_1=np.load('train_5class_1800.npy', allow_pickle=True, encoding='bytes')

In [3]:
train_ext=np.load('train_extra_added_200.npy', allow_pickle=True, encoding='bytes')

In [4]:
train=np.concatenate((train_1, train_ext), axis=0)

In [5]:
val=np.load('val_5class_200.npy', allow_pickle=True, encoding='bytes')

In [6]:
test=np.load('test_5class_500.npy', allow_pickle=True, encoding='bytes')

In [7]:
np.random.shuffle(train)
np.random.shuffle(test)
np.random.shuffle(val)

In [8]:
# preprocessing train data

n=len(train)
x_train = train[:, :-1]
y_train = train[:, -1]

g = []
for i in range(n):
    a=x_train[i][0]
    g.append(a)
    
train_x = np.array(g)

In [9]:
# preprocessing validation data

o=len(val)
x_val = val[:, :-1]
y_val = val[:, -1]

k=[]
for i in range(o):
    c=x_val[i][0]
    k.append(c)
    
val_x = np.array(k)

In [ ]:
#preprocessing test data

m=len(test)
x_test = test[:, :-1]
y_test = test[:, -1]

h=[]
for i in range(m):
    b=x_test[i][0]
    h.append(b)
    
test_x = np.array(h)


In [10]:
import category_encoders as ce
enc = ce.OneHotEncoder(y_train, handle_unknown='return_nan', return_df=True, use_cat_names=True)

In [11]:
y_train = enc.fit_transform(y_train)

In [12]:
y_val = enc.transform(y_val)

In [ ]:
y_test = enc.transform(y_test)

In [13]:
train_x.shape

(2000, 432, 504, 3)

In [14]:
cnn = Sequential()

cnn.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(432, 504, 3))) #1

cnn.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #2
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #3
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #4
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #5
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))


cnn.add(GlobalAveragePooling2D())

cnn.add(Dense(512, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(330, activation='relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(170, activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(40, activation='relu'))
cnn.add(Dropout(0.25))

cnn.add(Dense(5, activation='softmax'))

In [15]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 432, 504, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 432, 504, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 216, 252, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 216, 252, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 216, 252, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 108, 126, 128)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 108, 126, 128)     0

In [16]:
cnn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [17]:
filepath = "saved-model-{epoch:02d}-{val_loss:.2f}.hdf5"

In [18]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='max')

In [19]:
cnn.fit(train_x, y_train, batch_size=8, epochs=100, validation_data=(val_x, y_val), verbose=1, callbacks=[checkpoint])

Epoch 1/100
166/250 [==================>...........] - ETA: 1:43:11 - loss: nan - accuracy: 0.2108

KeyboardInterrupt: 